# Lab 1: State management

Table of contents

1. [Overview](#overview)
    1. [Features](#features)
1. [Prerequisites](#prerequisites)
1. [Common state operations](#common-state-operations)
    1. [Save a key/value pair](#save-a-key/value-pair)
    1. [Get value for a key](#get-value-for-a-key)
    1. [Get values for multiple keys](#get-values-for-multiple-keys)
    1. [Delete a key/value pair](#delete-a-key/value-pair)
    1. [(Optional) Use Redis CLI to view data](#optional-use-redis-cli-to-view-data)
1. [Clean up](#clean-up)
1. [Running the quickstart](#running-the-quickstart)
1. [Next steps](#next-steps)
    1. [References](#references)

## Overview

Your application can use Dapr’s state management API to save, read, and query **key/value pairs** in the supported state stores (such as Redis, Azure Cosmos DB, and Azure SQL Server). See the full list of supported state stores [here](https://docs.dapr.io/reference/components-reference/supported-state-stores/). Using a state store component, you can build stateful, long running applications that save and retrieve their state (like a shopping cart or a game’s session state).

<img src="../static/01-state-store.png" width="800">

In this lab, you will learn how to use the Dapr state management API to save and retrieve state.

### Features

With the state management API building block, your application can leverage features that are _typically complicated and error-prone to build_, including:
1. Concurrency control - Optimistic Concurrency Control (OCC) using ETags
1. Data consistency - Strong and Eventual consistency
1. Multi-read and multi-write - Bulk (read) or transactional (write, update, and delete) operations
1. Querying and filtering using key/value data
1. Automatic client encryption
1. Time-to-live (TTL) for state entries

## Prerequisites

1. [Install Dapr CLI](https://docs.dapr.io/getting-started/install-dapr-cli/).
1. Initialize Dapr in your local environment, run `dapr init` in your terminal.
1. Run the cell below to import some helper functions used in this lab.

In [ ]:
# Run this cell to import the necessary libraries
import json
import os
import sys

sys.path.append(os.path.abspath('../utils'))
from shell import execute, execute_async

If you have set up everything correctly, the following command should display:
```text
CLI version: x.y.z 
Runtime version: a.b.c
```

In [ ]:
print(execute("dapr --version"))

Next, run the Dapr process on port 3500.

In [ ]:
# Start a daprd process listening on port 3500
execute_async("dapr run --app-id state-lab-app --dapr-http-port 3500")

Validate that Dapr is running!

Expected output:
```text
  APP ID         HTTP PORT  GRPC PORT  APP PORT  COMMAND  AGE  CREATED  DAPRD PID  CLI PID  
  state-lab-app  3500       <random>      0               <X> <datetime>  <pid>     <pid>    
```

In [ ]:
print(execute("dapr list"))

## Common state operations

Now that you have setup Dapr, you can start using the state management API. We will be using cURL via Python to call Dapr APIs. Note, the Dapr CLI comes with a Redis state store component pre-installed, which is used in this lab.

### Save a key/value pair

In [ ]:
request_body = json.dumps([{"key":"dapr is", "value":"awesome"}])
execute(f'curl -X POST \
    -H "Content-Type: application/json" \
    -d \'{request_body}\' \
    http://localhost:3500/v1.0/state/statestore')

request_body = json.dumps([{"key":"another key", "value":"another value"}])
execute(f'curl -X POST \
    -H "Content-Type: application/json" \
    -d \'{request_body}\' \
    http://localhost:3500/v1.0/state/statestore')

### Get value for a key

Expected output:
```text
"awesome"
```

In [ ]:
print(execute("curl http://localhost:3500/v1.0/state/statestore/dapr%20is"))

### Get values for multiple keys

You can also query for multiple keys together using a bulk request.

In [ ]:
request_body = json.dumps({"keys": ["dapr is", "another key"]})
print(execute(f'curl -X POST \
    -H "Content-Type: application/json" \
    -d \'{request_body}\' \
    http://localhost:3500/v1.0/state/statestore/bulk'))

### Deleting a key/value pair

In [ ]:
execute(f'curl -X DELETE http://localhost:3500/v1.0/state/statestore/another%20key')

# Query the state store, it should be empty!
print(execute("curl http://localhost:3500/v1.0/state/statestore/another%20key"))

### (Optional) Use Redis CLI to view data

In your terminal, 
```bash
# Connect to the Redis docker container, and run the Redis CLI
docker exec -it $(docker ps | grep redis | awk '{print $1}') redis-cli
```
```bash
# List all keys
# Output: "state-lab-app||dapr is"
keys *

# Get the value for the key "dapr is", created by app "state-lab-app"
# Output:
# 1) "data"
# 2) "\"awesome\""
# 3) "version"
# 4) "1"
HGETALL "state-lab-app||dapr is"
```

## Clean up

Stop the Dapr process that we started earlier. Expected output:
```text
✅  app stopped successfully: state-lab-app
```

In [ ]:
print(execute("dapr stop --app-id state-lab-app"))

## Running the quickstart

Next, check out the quickstart from https://github.com/dapr/quickstarts/tree/master/state_management. There are multiple programming languages available, you can choose the one of your choice.

Within each quickstart, there are two flavors available
1. HTTP - Uses a simple HTTP client to invoke Dapr APIs (similar to what we did in this lab).
1. SDK - Uses special Dapr SDKs to call Dapr APIs. This makes it easier to write Dapr applications, as the SDKs abstract away the HTTP calls.

You have already seen how to make HTTP calls to invoke Dapr, so you can try out quickstarts using the SDKs. If you are interested in learning more about the Dapr SDKs, you can read more about them [here](https://docs.dapr.io/developing-applications/sdks).

Clone the quickstarts repo if you haven't already:

```bash
git clone https://github.com/dapr/quickstarts
cd quickstarts/state_management/python/sdk
```

To run the quickstart, you can follow the instructions from here https://github.com/dapr/quickstarts/tree/master/state_management/python/sdk.

## Next steps

Thank you for completing this lab! Please check out the references below to learn more.

### References

1. State management overview: https://docs.dapr.io/developing-applications/building-blocks/state-management/state-management-overview/
1. State management API reference: https://docs.dapr.io/reference/api/state_api/
1. State management quickstarts: https://github.com/dapr/quickstarts/tree/master/state_management
1. Dapr for dotnet developers article: https://learn.microsoft.com/en-us/dotnet/architecture/dapr-for-net-developers/state-management